In [1]:
import re
import pandas as pd
import numpy as np
import pickle
import os
import jieba
import jieba.posseg as pseg

In [2]:
#將繁體字進行切斷

In [3]:
data_path = "./dataset_6.csv"
data = pd.read_csv(data_path, engine='python')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73007 entries, 0 to 73006
Data columns (total 5 columns):
cbodyurl      73007 non-null object
title         73007 non-null object
txtcontent    71411 non-null object
label         73007 non-null int64
text          73007 non-null object
dtypes: int64(1), object(4)
memory usage: 2.8+ MB


In [5]:
data.head()

,cbodyurl,title,txtcontent,label,text
0,https://inetrsc.cupoy.com/http176.34.63.351173...,近看膚質也好到像自備濾鏡！曾之喬偷懶急救居然只靠這一招 - FG彩妝保養 - Fashion...,(圖片來源/FB@曾之喬)在還沒看到曾之喬以前，真的無法相信有人的膚質可以好到這種程度，大笑...,3,在還沒看到曾之喬以前，真的無法相信有人的膚質可以好到這種程度，大笑起來沒有細紋，膚質平滑，白...
1,https://inetrsc.cupoy.com/http176.34.63.351173...,夾睫毛總是夾到眼皮？睫毛長又翹「兩段式」夾、刷法技巧大公開 - FG彩妝保養 - Fashi...,(圖片來源：Pinterest/INSTAGRAM@kerina_hsueh)每次夾睫毛總是...,3,其實夾睫毛不是一件這麼難的事，只是有些步驟常常被忽略~快跟著以下步驟，讓妳就算不用種睫毛，也...
2,https://inetrsc.cupoy.com/http24h.pchome.com.t...,PChome線上購物 -,NaN,11,但猶豫期並非試用期，請留意，您所退回的商品必須回復原狀（須回復至商品到貨時的原始狀態）並且保...
3,https://inetrsc.cupoy.com/http24h.pchome.com.t...,PChome線上購物 -,NaN,11,但猶豫期並非試用期，請留意，您所退回的商品必須回復原狀（須回復至商品到貨時的原始狀態）並且保...
4,https://inetrsc.cupoy.com/http30plus.30.com.tw...,在學賺錢之前，先學會如何管理錢 - 【30雜誌】30+BLOG,即使找到金飯碗，領死薪水是不會致富的！人人稱羨的科技新貴，跳脫舒適圈創業去！人人都想賺錢、人...,6,」這樣的觀念從小就被植入，當時的我一直很想賺錢，學生時代幾乎都會去打工，像是：KTV服務生、...


In [8]:
!pip install opencc-python-reimplemented

In [9]:
from opencc import OpenCC
cc = OpenCC('s2tw')

data_raw = data['text'].tolist()
new_data = []

for i in data_raw:
    i = str(i).lower()
    new_data.append(cc.convert(i))
#     print(new_data)
    

#data_raw['tw_text'] = cc.convert(data_raw['text'])

In [10]:
data_model = data.drop(['cbodyurl','title','txtcontent','text'], axis = 1)
data_model.head()

,label
0,3
1,3
2,11
3,11
4,6


In [11]:
data_model['text'] = new_data
data_model.head()

,label,text
0,3,在還沒看到曾之喬以前，真的無法相信有人的膚質可以好到這種程度，大笑起來沒有細紋，膚質平滑，白...
1,3,其實夾睫毛不是一件這麼難的事，只是有些步驟常常被忽略~快跟著以下步驟，讓妳就算不用種睫毛，也...
2,11,但猶豫期並非試用期，請留意，您所退回的商品必須回復原狀（須回復至商品到貨時的原始狀態）並且保...
3,11,但猶豫期並非試用期，請留意，您所退回的商品必須回復原狀（須回復至商品到貨時的原始狀態）並且保...
4,6,」這樣的觀念從小就被植入，當時的我一直很想賺錢，學生時代幾乎都會去打工，像是：ktv服務生、...


In [ ]:
data.to_csv('dataset_8_multi.csv',index=False)

In [64]:
# data_path = "./dataset_7.csv"
# data_model = pd.read_csv(data_path, engine='python')

In [77]:
data_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73007 entries, 0 to 73006
Data columns (total 2 columns):
label    73007 non-null int64
text     73007 non-null object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [12]:
#處理label
import re
import sys
label_idxs = []
label_vs = []
for index, article in data_model.iterrows():
    label_str = str(article['label'])
    label_arr = re.split(r'\t+', label_str)
    label_arr = [int(l) for l in label_arr]
    label_idxs.append(index)
    
    v = np.zeros(14).astype('int32')
    for li in label_arr:
        v[li] = 1
    label_vs.append(v)

In [13]:
column_list = range(14)

pre_res = ["cat_" + str(sub) for sub in column_list]


res = pd.DataFrame (label_vs,columns=pre_res)
res

,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73002,0,0,0,0,0,0,0,0,0,1,0,0,0,0
73003,0,0,0,0,0,0,0,0,0,1,0,0,0,0
73004,0,0,0,0,0,0,0,0,0,1,0,0,0,0
73005,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
from sklearn.model_selection import train_test_split
# train, test = train_test_split(data_model, test_size=0.3, random_state=86)

In [15]:
train_label = res

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(data_model,train_label, test_size=0.2, random_state=86, shuffle=True)

In [17]:
X_train.shape

(58405, 2)

In [18]:
X_test.shape

(14602, 2)

In [80]:
Y_train.shape

(58405, 14)

In [19]:
stop_path = './jieba/stop_words2.txt'

In [20]:
def preprocess_sentences(data, colname):
    sentences = []
    data0 = data[colname].tolist()
    stop_words = open(stop_path).read().splitlines()
    for i, text in enumerate(data0):
        line = []
        text = str(text)
        for w in jieba.cut(text, cut_all=False):

            # remove stopwords and digits
            # can define your own rules
            if w not in stop_words and not bool(re.match('[0-9\n●「」\r\n -/`【】]+', w)):
                line.append(w)

        sentences.append(line)

        if i%10000==0:
            print(i, '/', len(data))
    return sentences

In [21]:
X_train_sen = preprocess_sentences(X_train, 'text')

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.380 seconds.
Prefix dict has been built succesfully.


0 / 58405
10000 / 58405
20000 / 58405
30000 / 58405
40000 / 58405
50000 / 58405


In [22]:
X_test_sen = preprocess_sentences(X_test, 'text')

0 / 14602
10000 / 14602


In [23]:
# save data as pickle format
with open("storeSpace/cupoy_cutted_X_train_single", "wb") as file:
    pickle.dump(X_train_sen, file)

In [24]:
# save data as pickle format
with open("storeSpace/cupoy_cutted_X_test_single", "wb") as file:
    pickle.dump(X_test_sen, file)

In [ ]:
#Word2Vec

In [25]:
import pickle
from gensim.models import word2vec
import random
import logging
import os


In [26]:
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

In [27]:
## load 'article_cutted'
with open('storeSpace/cupoy_cutted_X_train_single', 'rb') as file:
    cupoy_cutted_X_train = pickle.load(file)

In [28]:
## load 'article_cutted'
with open('storeSpace/cupoy_cutted_X_test_single', 'rb') as file:
    cupoy_cutted_X_test = pickle.load(file)

In [ ]:
#Averaging Word Embedding for Each Doc

In [29]:
import pickle
from gensim.models import Doc2Vec, doc2vec
from gensim.models import word2vec
from gensim.models import KeyedVectors
import gensim
import random
import numpy as np
import logging
import os

In [30]:
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

In [31]:
# load 'article_cutted'
with open('storeSpace/cupoy_cutted_X_train_single', 'rb') as file:
    cupoy_cutted_X_train = pickle.load(file)

In [32]:
with open('storeSpace/cupoy_cutted_X_test_single', 'rb') as file:
    cupoy_cutted_X_test = pickle.load(file)

In [33]:
load_wiki = KeyedVectors.load_word2vec_format('word2vec_model/zh_wiki_word2vec_300.txt')

2020-08-29 11:39:26,935: INFO: loading projection weights from word2vec_model/zh_wiki_word2vec_300.txt
/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-29 11:44:09,127: INFO: loaded (614083, 300) matrix from word2vec_model/zh_wiki_word2vec_300.txt


In [34]:
load_wiki.most_similar(positive=['英文'])

2020-08-29 11:44:09,135: INFO: precomputing L2-norms of word weight vectors


[('中文', 0.7536253929138184),
 ('法文', 0.6874601244926453),
 ('日文', 0.6220279932022095),
 ('西班牙文', 0.6084370613098145),
 ('韓文', 0.5994431972503662),
 ('德文', 0.5884890556335449),
 ('俄文', 0.5821559429168701),
 ('譯名', 0.5794787406921387),
 ('芬蘭文', 0.5641646981239319),
 ('英文名', 0.5599370002746582)]

In [ ]:
# from wikipedia2vec import Wikipedia2Vec
# wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

參考文獻：
用維基語料訓練 Word2vec 和 FastText embedding
https://medium.com/@black_swan/%E7%94%A8%E7%B6%AD%E5%9F%BA%E8%AA%9E%E6%96%99%E8%A8%93%E7%B7%B4-word2vec-%E5%92%8C-fasttext-embedding-25ede5b15994
Gensim:
https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.load
Pretrained Embeddings
https://wikipedia2vec.github.io/wikipedia2vec/pretrained/



In [35]:
# filter words that not in word2vec's vocab
train_data_filtered = [[w for w in l if w in load_wiki.wv] for l in cupoy_cutted_X_train]
test_data_filtered = [[w for w in l if w in load_wiki.wv] for l in cupoy_cutted_X_test]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
# compute average word vector
def avg_vec(data_filtered,model):
    avg_vector = []

    for l in data_filtered:
        if len(l)==0:
            avg_vector.append(np.array([0]*300))
        else:
            avg_vector.append(np.mean([model.wv[w] for w in l], axis=0))
    return avg_vector

In [37]:
train_avg_vec = avg_vec(train_data_filtered, load_wiki)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [38]:
test_avg_vec = avg_vec(test_data_filtered, load_wiki)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [39]:
# save result
with open('storeSpace/train_avg_vec_byWiki_single', 'wb') as file:
    pickle.dump(train_avg_vec, file)

In [40]:
# save result
with open('storeSpace/test_avg_vec_byWiki_single', 'wb') as file:
    pickle.dump(test_avg_vec, file)

In [41]:
train_avg_df = pd.DataFrame(train_avg_vec)

In [42]:
test_avg_df = pd.DataFrame(test_avg_vec)

In [43]:
column_list = range(300)
col_name = ["char_" + str(sub) for sub in column_list]

In [44]:
train_avg_df.columns = col_name
test_avg_df.columns = col_name

In [45]:
train_avg_df

,char_0,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,...,char_290,char_291,char_292,char_293,char_294,char_295,char_296,char_297,char_298,char_299
0,-0.033709,0.242503,-0.171803,-0.023677,0.076615,0.014258,-0.055446,0.003849,-0.030292,0.053203,...,-0.003379,0.020401,0.076694,0.018111,0.000624,0.009766,0.054347,-0.064101,0.056249,0.068927
1,-0.007201,0.256056,-0.150317,-0.036970,0.069526,0.029032,-0.075840,0.127239,0.043852,0.154765,...,-0.021832,0.027505,0.080909,0.054135,-0.003762,0.075207,0.070294,-0.077187,0.054489,0.069343
2,-0.046384,0.232557,-0.062982,0.003217,0.016654,-0.056337,-0.006642,0.061128,-0.025081,0.083010,...,-0.040497,-0.073900,0.009410,0.076790,-0.021548,0.063108,0.028991,-0.055256,0.066123,0.110284
3,-0.003068,0.253867,-0.021721,0.036688,0.077250,-0.033863,0.054679,-0.056870,-0.045017,0.174222,...,-0.082848,0.056733,0.028177,0.105787,0.032486,0.070363,0.071506,-0.149105,0.081617,0.042613
4,0.028769,0.156787,-0.060777,0.020189,0.032972,0.005192,0.009878,0.013911,0.103503,0.062920,...,-0.088808,0.026080,0.103571,0.029077,-0.002270,-0.017608,0.042008,-0.041048,0.127885,0.060874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58400,0.003164,0.223701,-0.069305,-0.040563,0.024109,-0.019927,0.000992,0.112055,0.044526,0.013215,...,-0.014902,-0.012244,0.045194,0.013524,-0.000574,-0.045446,0.053148,-0.040465,0.079521,0.049010
58401,-0.010187,0.236410,-0.079693,-0.084997,0.023736,0.046709,-0.032983,0.021336,0.040288,0.115151,...,-0.034822,0.029059,0.036163,0.073927,0.092462,0.077456,0.058480,-0.071883,0.114570,0.089503
58402,-0.035849,0.222425,-0.046319,-0.050833,-0.001862,-0.055264,-0.070974,0.002338,0.046020,-0.055583,...,-0.029715,-0.019601,0.091913,0.013811,0.106003,0.018668,0.073467,-0.106612,0.005641,0.054428
58403,-0.011308,0.272369,-0.036697,0.023074,-0.004903,0.012918,-0.085521,0.004318,0.022689,0.069498,...,-0.000488,-0.039147,0.149024,0.030676,0.028170,0.062823,0.121045,-0.035058,0.000499,0.053418


In [ ]:
#訓練模型前記得切資料

In [78]:
X_train.shape

(58405, 2)

In [79]:
X_test.shape

(14602, 2)

In [ ]:
#模型訓練

In [46]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [82]:
def single_label_model(col):
    Y_train_ = Y_train[col]
    xgb_model=XGBClassifier(objective='binary:logistic',learning_rate=0.2,min_child_weight=1,max_depth=6)
    xgb_model.fit(train_avg_df,Y_train_,eval_metric='merror')
    return xgb_model

參考文獻：

Multiclass & Multilabel Classification with XGBoost
https://medium.com/@gabrielziegler3/multiclass-multilabel-classification-with-xgboost-66195e4d9f2d
使用 scikit-learn 實現多類別及多標籤分類算法
https://www.cnblogs.com/Allen-rg/p/9492303.html

In [99]:
def single_label_predict(model,testdf,col):
    from sklearn.metrics import accuracy_score, hamming_loss
    import matplotlib.pyplot as plt
    from matplotlib.pylab import rcParams
    Y_test_ = Y_test[col]
    y_pred = model.predict(testdf)
    accuracy = accuracy_score(Y_test_, y_pred) * 100.0
    HL = hamming_loss(Y_test_, y_pred)
    return accuracy, HL
    
    

In [55]:
Y_train['cat_0'].value_counts()

0    55898
1     2507
Name: cat_0, dtype: int64

In [81]:
Y_train_ = Y_train['cat_0']

xgb_model=XGBClassifier(objective='binary:logistic',learning_rate=0.2,min_child_weight=1,max_depth=6)
xgb_model.fit(train_avg_df,Y_train_,eval_metric='merror')

# model0 = single_label_model('cat_0')


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [100]:
accuracy0 = single_label_predict(xgb_model,test_avg_df,'cat_0' )
print(accuracy0)

(98.71935351321737, 0.012806464867826325)


In [103]:
# model1 = single_label_model('cat_1')
accuracy1 = single_label_predict(model1,test_avg_df,'cat_1' )
print(accuracy1)

(97.04150116422407, 0.02958498835775921)


In [104]:
# model2 = single_label_model('cat_2')
accuracy2 = single_label_predict(model2,test_avg_df,'cat_2' )
print(accuracy2)

(97.69894534995206, 0.023010546500479387)


In [105]:
# model3 = single_label_model('cat_3')
accuracy3 = single_label_predict(model3,test_avg_df,'cat_3' )
print(accuracy3)

(98.72620189015203, 0.01273798109847966)


In [106]:
# model4 = single_label_model('cat_4')
accuracy4 = single_label_predict(model4,test_avg_df,'cat_4' )
print(accuracy4)

(98.69880838241338, 0.01301191617586632)


In [107]:
# model5 = single_label_model('cat_5')
accuracy5 = single_label_predict(model5,test_avg_df,'cat_5' )
print(accuracy5)

(98.18518011231339, 0.01814819887686618)


In [108]:
# model6 = single_label_model('cat_6')
accuracy6 = single_label_predict(model6,test_avg_df,'cat_6' )
print(accuracy6)

(96.2813313244761, 0.03718668675523901)


In [109]:
# model7 = single_label_model('cat_7')
accuracy7 = single_label_predict(model7,test_avg_df,'cat_7' )
print(accuracy7)

(98.5070538282427, 0.014929461717572935)


In [110]:
# model8 = single_label_model('cat_8')
accuracy8 = single_label_predict(model8,test_avg_df,'cat_8' )
print(accuracy8)

(94.43226955211614, 0.055677304478838514)


In [111]:
# model9 = single_label_model('cat_9')
accuracy9 = single_label_predict(model9,test_avg_df,'cat_9' )
print(accuracy9)

(97.6510067114094, 0.02348993288590604)


In [112]:
# model10 = single_label_model('cat_10')
accuracy10 = single_label_predict(model10,test_avg_df,'cat_10' )
print(accuracy10)

(97.04834954115874, 0.029516504588412547)


In [113]:
# model11 = single_label_model('cat_11')
accuracy11 = single_label_predict(model11,test_avg_df,'cat_11' )
print(accuracy11)

(95.1171072455828, 0.04882892754417203)


In [114]:
# model12 = single_label_model('cat_12')
accuracy12 = single_label_predict(model12,test_avg_df,'cat_12' )
print(accuracy12)

(98.17148335844405, 0.018285166415559512)


In [115]:
# model13 = single_label_model('cat_13')
accuracy13 = single_label_predict(model13,test_avg_df,'cat_13' )
print(accuracy13)

(98.91110806738803, 0.01088891932611971)
